<a href="https://colab.research.google.com/github/eshiz/apostila_python/blob/master/detecFreque.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install soundfile --quiet

import os
import shutil
import numpy as np
import pandas as pd
import soundfile as sf
import matplotlib.pyplot as plt
from IPython.display import display
from google.colab import files, drive
import math

In [2]:
SAMPLE_WINDOW = 4096        # janela FFT; aumentar -> melhor resolução em freq baixa
HPS_HARMONICS = [2,3,4]     # harmônicos para HPS
MAX_FREQ_PLOT = 2000        # limite para plot de espectro
USE_HPS = True              # usar HPS (recomendado para cordas)
TRIM_SILENCE = True         # remover silêncio inicial
TRIM_THRESHOLD = 0.02       # fração da amplitude máxima para considerar som
MIN_SILENCE_SEC = 0.02      # mínimo de silêncio a ser considerado
AUTOCORR_FALLBACK = True    # usar autocorrelação quando FFT/HPS der resultado estranho
MIN_ACCEPTABLE_FREQ = 20.0  # se detectar < isso, considerar inválido e usar fallback

In [3]:
print("Escolha uma opção: (1) Upload direto de arquivos; (2) Montar Google Drive e apontar para a pasta.")
modo = input("Digite 1 para upload ou 2 para Drive (press Enter -> 1): ")
if modo.strip() == "" or modo.strip() == "1":
    print("Faça upload dos arquivos .wav e do CSV (se tiver) — selecione múltiplos arquivos na janela que abrir.")
    uploaded = files.upload()  # selecione seus 18 wav + csv
    # mover arquivos carregados para /content/gravacoes
    os.makedirs("/content/gravacoes", exist_ok=True)
    for name in uploaded.keys():
        shutil.move(name, os.path.join("/content/gravacoes", name))
    gravacoes_folder = "/content/gravacoes"
    print("Upload concluído. Arquivos movidos para /content/gravacoes")
else:
    drive.mount('/content/drive')
    gravacoes_folder = input("Cole o caminho da pasta no Drive contendo os arquivos (ex: /content/drive/MyDrive/gravacoes): ").strip()
    if not os.path.exists(gravacoes_folder):
        raise FileNotFoundError(f"Pasta não encontrada: {gravacoes_folder}")
    print("Usando pasta no Drive:", gravacoes_folder)

# listar arquivos encontrados
wav_files = sorted([f for f in os.listdir(gravacoes_folder) if f.lower().endswith(".wav")])
print(f"{len(wav_files)} .wav encontrados em {gravacoes_folder}")
for f in wav_files:
    print(" -", f)

Escolha uma opção: (1) Upload direto de arquivos; (2) Montar Google Drive e apontar para a pasta.
Digite 1 para upload ou 2 para Drive (press Enter -> 1): 1
Faça upload dos arquivos .wav e do CSV (se tiver) — selecione múltiplos arquivos na janela que abrir.


Saving A2(1).wav to A2(1).wav
Saving A2(2).wav to A2(2).wav
Saving A2(3).wav to A2(3).wav
Saving B3(1).wav to B3(1).wav
Saving B3(2).wav to B3(2).wav
Saving B3(3).wav to B3(3).wav
Saving D3(1).wav to D3(1).wav
Saving D3(2).wav to D3(2).wav
Saving D3(3).wav to D3(3).wav
Saving Dados - Página1.csv to Dados - Página1.csv
Saving E2(1).wav to E2(1).wav
Saving E2(2).wav to E2(2).wav
Saving E2(3).wav to E2(3).wav
Saving E4(1).wav to E4(1).wav
Saving E4(2).wav to E4(2).wav
Saving E4(3).wav to E4(3).wav
Saving G3(1).wav to G3(1).wav
Saving G3(2).wav to G3(2).wav
Saving G3(3).wav to G3(3).wav
Upload concluído. Arquivos movidos para /content/gravacoes
18 .wav encontrados em /content/gravacoes
 - A2(1).wav
 - A2(2).wav
 - A2(3).wav
 - B3(1).wav
 - B3(2).wav
 - B3(3).wav
 - D3(1).wav
 - D3(2).wav
 - D3(3).wav
 - E2(1).wav
 - E2(2).wav
 - E2(3).wav
 - E4(1).wav
 - E4(2).wav
 - E4(3).wav
 - G3(1).wav
 - G3(2).wav
 - G3(3).wav


In [4]:
def trim_silence(x, fs, threshold=TRIM_THRESHOLD, min_silence_sec=MIN_SILENCE_SEC):
    """Remove silêncio inicial com base em threshold relativo à amplitude máxima."""
    if x is None or len(x) == 0:
        return x
    abs_x = np.abs(x)
    max_amp = np.max(abs_x)
    if max_amp == 0:
        return x
    thresh = threshold * max_amp
    idxs = np.where(abs_x > thresh)[0]
    if len(idxs) == 0:
        return x
    start = max(0, idxs[0] - int(0.005 * fs))  # 5ms de folga
    return x[start:]

def detect_frequency_fft(x, fs, N=SAMPLE_WINDOW):
    """Detecta frequência por FFT simples (pico de magnitude). Retorna (f0, freqs, mag)."""
    if len(x) < 1:
        return np.nan, None, None
    x_seg = x[:N]
    if len(x_seg) < N:
        x_seg = np.pad(x_seg, (0, N - len(x_seg)))
    window = np.hanning(N)
    X = np.fft.rfft(x_seg * window)
    freqs = np.fft.rfftfreq(N, 1/fs)
    mag = np.abs(X)
    idx = np.argmax(mag)
    return freqs[idx], freqs, mag

def detect_frequency_hps(x, fs, N=SAMPLE_WINDOW, harmonics=HPS_HARMONICS):
    """Detecta frequência usando HPS (multiplica espectros decimados)."""
    f0, freqs, mag = detect_frequency_fft(x, fs, N)
    if mag is None:
        return np.nan, freqs, mag, None
    hps = mag.copy()
    for h in harmonics:
        dec = mag[::h]
        if len(dec) == 0:
            continue
        hps[:len(dec)] *= dec
    idx = np.argmax(hps)
    return freqs[idx], freqs, mag, hps

def autocorr_pitch(x, fs, fmin=40, fmax=1000):
    """Autocorrelation-based pitch detection (returns frequency or nan)."""
    if len(x) == 0:
        return np.nan
    # autocorrelação no segmento
    x = x - np.mean(x)
    corr = np.correlate(x, x, mode='full')
    corr = corr[len(corr)//2:]
    # evitar 0-lag forte
    # procurar primeiro pico significativo após o lag correspondente a fmax
    min_lag = int(fs / fmax) if fmax>0 else 1
    max_lag = int(fs / fmin) if fmin>0 else len(corr)-1
    if max_lag >= len(corr):
        max_lag = len(corr)-1
    corr_segment = corr[min_lag:max_lag+1]
    if len(corr_segment) == 0:
        return np.nan
    idx_rel = np.argmax(corr_segment)
    lag = idx_rel + min_lag
    if lag == 0:
        return np.nan
    return fs / lag

def plot_spectrum(freqs, mag, title="", max_freq=MAX_FREQ_PLOT):
    if freqs is None or mag is None:
        print("Sem dados para plot:", title)
        return
    plt.figure(figsize=(9,3))
    plt.plot(freqs, mag)
    plt.xlim(0, max_freq)
    plt.xlabel("Frequência (Hz)")
    plt.ylabel("Magnitude")
    plt.title(title)
    plt.grid(True)
    plt.show()

In [5]:
csv_name = "Dados - Página1.csv"  # altere se seu CSV tiver outro nome
csv_path = os.path.join(gravacoes_folder, csv_name)

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print("CSV carregado:", csv_path, "(", len(df), "linhas )")
else:
    print("CSV não encontrado em", csv_path, "- criarei DataFrame a partir dos .wav.")
    df = pd.DataFrame({"arquivo": wav_files})

# garantir coluna 'arquivo' limpa (apenas nome do arquivo)
if "arquivo" not in df.columns:
    df["arquivo"] = wav_files
else:
    df["arquivo"] = df["arquivo"].apply(lambda x: os.path.basename(str(x)))

# verificar existência física de cada arquivo listado
missing = [f for f in df["arquivo"].unique() if not os.path.exists(os.path.join(gravacoes_folder, f))]
if missing:
    print("ATENÇÃO - arquivos listados no CSV NÃO encontrados na pasta:")
    for m in missing:
        print(" -", m)
    # filtrar só os existentes
    df = df[df["arquivo"].apply(lambda x: os.path.exists(os.path.join(gravacoes_folder, x)))].reset_index(drop=True)

display(df.head(20))

CSV carregado: /content/gravacoes/Dados - Página1.csv ( 18 linhas )


,Arquivo,Corda,Nota,Freq_teorica,Freq_detectada,erro_Hz,obs,arquivo
0,E2(1).wav,6,E2,82.41,NaN,NaN,NaN,A2(1).wav
1,E2(2).wav,6,E2,82.41,NaN,NaN,NaN,A2(2).wav
2,E2(3).wav,6,E2,82.41,NaN,NaN,NaN,A2(3).wav
3,A2(1).wav,5,A2,110.00,NaN,NaN,NaN,B3(1).wav
4,A2(2).wav,5,A2,110.00,NaN,NaN,NaN,B3(2).wav
5,A2(3).wav,5,A2,110.00,NaN,NaN,NaN,B3(3).wav
6,D3(1).wav,4,D3,146.83,NaN,NaN,NaN,D3(1).wav
7,D3(2).wav,4,D3,146.83,NaN,NaN,NaN,D3(2).wav
8,D3(3).wav,4,D3,146.83,NaN,NaN,NaN,D3(3).wav
9,G3(1).wav,3,G3,196.00,NaN,NaN,NaN,E2(1).wav


In [6]:
info = []
for f in df['arquivo']:
    p = os.path.join(gravacoes_folder, f)
    try:
        info_wav = sf.info(p)
        dur = info_wav.duration
        sr = info_wav.samplerate
    except Exception as e:
        dur = None; sr = None
    info.append({"arquivo": f, "duration_s": dur, "samplerate": sr})
info_df = pd.DataFrame(info)
display(info_df.sort_values(by="duration_s"))
print("Procure arquivos com duration_s muito pequeno ou samplerate estranho (ex: 0).")

,arquivo,duration_s,samplerate
2,A2(3).wav,4.911020,44100
1,A2(2).wav,4.957460,44100
6,D3(1).wav,4.957460,44100
12,E4(1).wav,4.969070,44100
17,G3(3).wav,4.969070,44100
15,G3(1).wav,4.980680,44100
4,B3(2).wav,4.980680,44100
5,B3(3).wav,4.992290,44100
10,E2(2).wav,4.992290,44100
8,D3(3).wav,4.992290,44100


Procure arquivos com duration_s muito pequeno ou samplerate estranho (ex: 0).


In [7]:
results = []

for i, row in df.iterrows():
    filename = row["arquivo"]
    path = os.path.join(gravacoes_folder, filename)
    print(f"[{i+1}/{len(df)}] Processando: {filename}")
    try:
        x, fs = sf.read(path)
        # garantir mono
        if x is None or len(x)==0:
            print("  -> ARQUIVO VAZIO ou NÃO LIDO")
            results.append({"arquivo": filename, "freq_detectada": np.nan, "fs": fs if 'fs' in locals() else None, "note": None, "erro_Hz": None})
            continue
        if x.ndim > 1:
            x = x[:,0]
        # normalizar
        max_abs = np.max(np.abs(x)) if x.size>0 else 0
        if max_abs > 0:
            x = x / max_abs
        # trim silêncio
        if TRIM_SILENCE:
            x = trim_silence(x, fs, threshold=TRIM_THRESHOLD, min_silence_sec=MIN_SILENCE_SEC)
        # detectar via HPS/FFT
        fdet = np.nan; freqs = None; mag = None; hps = None
        try:
            if USE_HPS:
                fdet, freqs, mag, hps = detect_frequency_hps(x, fs, N=SAMPLE_WINDOW, harmonics=HPS_HARMONICS)
            else:
                fdet, freqs, mag = detect_frequency_fft(x, fs, N=SAMPLE_WINDOW)
        except Exception as e:
            print("  -> erro na detecção FFT/HPS:", e)
            fdet = np.nan

        # validar resultado; se inválido ou muito baixo, usar autocorrelação como fallback
        invalid = False
        if fdet is None or (isinstance(fdet, float) and (math.isnan(fdet) or fdet < MIN_ACCEPTABLE_FREQ)):
            invalid = True

        if invalid and AUTOCORR_FALLBACK:
            f_aut = autocorr_pitch(x, fs, fmin=MIN_ACCEPTABLE_FREQ, fmax=2000)
            if not (f_aut is None or math.isnan(f_aut)):
                print(f"  -> fallback autocorr: {f_aut:.2f} Hz")
                fdet = f_aut
            else:
                print("  -> autocorr também não encontrou pitch válido")

        # se ainda inválido, registrar NaN e salvar espectro para inspeção
        if fdet is None or (isinstance(fdet, float) and math.isnan(fdet)):
            print("  -> freq_detectada inválida; marcar como NaN")
            entry = {"arquivo": filename, "freq_detectada": np.nan, "fs": int(fs), "freq_teorica": row.get("freq_teorica", None), "erro_Hz": None}
        else:
            entry = {"arquivo": filename, "freq_detectada": float(fdet), "fs": int(fs)}
            if "freq_teorica" in df.columns and not pd.isna(row.get("freq_teorica")):
                try:
                    ft = float(row["freq_teorica"])
                    entry["freq_teorica"] = ft
                    entry["erro_Hz"] = abs(entry["freq_detectada"] - ft)
                except:
                    entry["freq_teorica"] = None
                    entry["erro_Hz"] = None

        results.append(entry)

        # mostrar plot para os casos problemáticos ou para inspeção ocasional
        # Se frequencia detectada é muito baixa ou muito alta, plote para ver
        if math.isnan(entry["freq_detectada"]) or entry["freq_detectada"] < 40 or entry["freq_detectada"] > 2000:
            print("  -> abrindo plots (espectro e HPS) para inspeção")
            if freqs is not None and mag is not None:
                plot_spectrum(freqs, mag, title=f"{filename} FFT", max_freq=MAX_FREQ_PLOT)
            if hps is not None:
                plot_spectrum(freqs, hps, title=f"{filename} HPS", max_freq=MAX_FREQ_PLOT)
    except Exception as e:
        print(f"  -> ERRO irreversível processando {filename}: {e}")
        results.append({"arquivo": filename, "freq_detectada": np.nan, "fs": None, "freq_teorica": row.get("freq_teorica", None), "erro_Hz": None})

# montar DataFrame final
df_res = pd.DataFrame(results)
df_final = df.merge(df_res, on="arquivo", how="left")

print("\nProcessamento finalizado.")
print("Total arquivos:", len(df_final))
print("Detectados (não NaN):", df_final['freq_detectada'].count())

display(df_final.head(30))

# salvar CSV temporário no workspace
tmp_out = os.path.join("/content", "ATV2_resultados_detectados_temp.csv")
df_final.to_csv(tmp_out, index=False)
print("CSV temporário salvo em:", tmp_out)

# Célula 8 — salvar CSV final na pasta de gravações e oferecer download
out_csv = os.path.join(gravacoes_folder, "ATV2_resultados_detectados.csv")
df_final.to_csv(out_csv, index=False)
print("CSV final salvo em:", out_csv)

# forçar download pelo navegador
files.download(out_csv)

[1/18] Processando: A2(1).wav
  -> fallback autocorr: 109.16 Hz
[2/18] Processando: A2(2).wav
  -> fallback autocorr: 108.89 Hz
[3/18] Processando: A2(3).wav
  -> fallback autocorr: 108.89 Hz
[4/18] Processando: B3(1).wav
[5/18] Processando: B3(2).wav
  -> fallback autocorr: 245.00 Hz
[6/18] Processando: B3(3).wav
[7/18] Processando: D3(1).wav
  -> fallback autocorr: 146.03 Hz
[8/18] Processando: D3(2).wav
  -> fallback autocorr: 146.03 Hz
[9/18] Processando: D3(3).wav
  -> fallback autocorr: 146.51 Hz
[10/18] Processando: E2(1).wav
  -> fallback autocorr: 82.43 Hz
[11/18] Processando: E2(2).wav
  -> fallback autocorr: 82.28 Hz
[12/18] Processando: E2(3).wav
  -> fallback autocorr: 82.28 Hz
[13/18] Processando: E4(1).wav
  -> fallback autocorr: 164.55 Hz
[14/18] Processando: E4(2).wav
  -> fallback autocorr: 329.10 Hz
[15/18] Processando: E4(3).wav
[16/18] Processando: G3(1).wav
  -> fallback autocorr: 196.00 Hz
[17/18] Processando: G3(2).wav
  -> fallback autocorr: 196.00 Hz
[18/18] P

,Arquivo,Corda,Nota,Freq_teorica,Freq_detectada,erro_Hz,obs,arquivo,freq_detectada,fs
0,E2(1).wav,6,E2,82.41,NaN,NaN,NaN,A2(1).wav,109.158416,44100
1,E2(2).wav,6,E2,82.41,NaN,NaN,NaN,A2(2).wav,108.888889,44100
2,E2(3).wav,6,E2,82.41,NaN,NaN,NaN,A2(3).wav,108.888889,44100
3,A2(1).wav,5,A2,110.00,NaN,NaN,NaN,B3(1).wav,247.631836,44100
4,A2(2).wav,5,A2,110.00,NaN,NaN,NaN,B3(2).wav,245.000000,44100
5,A2(3).wav,5,A2,110.00,NaN,NaN,NaN,B3(3).wav,247.631836,44100
6,D3(1).wav,4,D3,146.83,NaN,NaN,NaN,D3(1).wav,146.026490,44100
7,D3(2).wav,4,D3,146.83,NaN,NaN,NaN,D3(2).wav,146.026490,44100
8,D3(3).wav,4,D3,146.83,NaN,NaN,NaN,D3(3).wav,146.511628,44100
9,G3(1).wav,3,G3,196.00,NaN,NaN,NaN,E2(1).wav,82.429907,44100


CSV temporário salvo em: /content/ATV2_resultados_detectados_temp.csv
CSV final salvo em: /content/gravacoes/ATV2_resultados_detectados.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>